# Mediapipe para detección de caras

In [ ]:
# Fuente https://github.com/googlesamples/mediapipe/blob/main/examples/face_landmarker/python/[MediaPipe_Python_Tasks]_Face_Landmarker.ipynb
# Utilidades visualización para el demostrador Face landmark detection

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [ ]:
# Carga de módulos
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Crea objeto FaceLandmarker 
# Obtener archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='C:/Users/otsed/Desktop/Docencia/VC/Modelos/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

vid = cv2.VideoCapture(0)

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()  
    if ret:
        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        detection_result = detector.detect(image)

        # Visualzia resultado
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # Visualziar blendshapes con gráfico de barras
        #plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
        # Muestra fotograma
        cv2.imshow('Vid', annotated_image)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

# Mediapipe para detección de manos
Si bien es código tomado del respositorio, no va muy bien, detecta una única mano y no siempre, la demo online va mejor

In [ ]:
# Fuente https://github.com/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb
# Utilidades visualización para el demostrador Face landmark detection

import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# Cámara
vid = cv2.VideoCapture(0)

# Crea objeto FaceLandmarker 
# Obtener archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='C:/Users/otsed/Desktop/Docencia/VC/Modelos/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)
  
while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:          
        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        detection_result = detector.detect(image)

        # Visualiza resultado
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # Muestra fotograma
        cv2.imshow('Vid', annotated_image)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Ejemplo alternativo, usando cvzone para la detección de manos con Mediapipe

In [ ]:
from cvzone.HandTrackingModule import HandDetector
import cv2

# Cámara
cap = cv2.VideoCapture(0)

# Initializa detector
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()

    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()

    # FBúsqueda de manos
    # 'draw' a True indica si se dibujan sobre la imagen 
    # 'flipType' a True para tratar la imagen reflejada
    hands, frame = detector.findHands(frame, draw=True, flipType=True)

    # Si hay manos detectadas
    if hands:
        # datos primera mano
        hand1 = hands[0]  
        lmList1 = hand1["lmList"]  # 21 landmarks
        bbox1 = hand1["bbox"]  # Contenedor (x,y,w,h)
        center1 = hand1['center']  # Centro
        handType1 = hand1["type"]  # identifica si es la mano derecha o izquierda

        # Contabiliza dedos extendidos de la mano
        fingers1 = detector.fingersUp(hand1)
        print(f'H1 = {fingers1.count(1)}', end=" ")  

        # Calcula distancia entre landmarks específicas, y lo dibuja
        length, info, img = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], img, color=(255, 0, 255),
                                                  scale=10)

        # Segunda mano presente?
        if len(hands) == 2:
            # Datos segunda mano
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Contabiliza dedos extendidos de la mano
            fingers2 = detector.fingersUp(hand2)
            print(f'H2 = {fingers2.count(1)}', end=" ")

            # CCalcula distancia entre landmarks específicas, y lo dibuja
            length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img, color=(255, 0, 0),
                                                      scale=10)

        print("\n")  

    # Display the image in a window
    cv2.imshow("Image", frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

# Modelos VQA de BLIP-2 

In [ ]:
import cv2
import torch
import os
from PIL import Image
from lavis.models import load_model_and_preprocess
from PIL import Image

#Por un error que aparecía https://github.com/explosion/spaCy/issues/7664
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Utiliza la gpu si está disponible (y habilitada)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carga modelo VQA
#Modelo OPT
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip_vqa", model_type="vqav2", is_eval=True, device=device)
#Modelo FLANT5XL, descarga modelos y he necesitado una 3080 paar ejecutar
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5", model_type="pretrain_flant5xl", is_eval=True, device=device)

Keepasking = True

#Imagen a cargar
#img = cv2.imread("images/Opel-Astra.png")
img = cv2.imread("images/FERsv3.png")

#Convierte a RGB y la muestra
converted = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
raw_image = Image.fromarray(converted)
cv2.imshow('Imagen', img)
cv2.waitKey(50)

while Keepasking:
    print('> Introduce una pregunta en inglés (exit para terminar)>')
    question = str(input())

    if question != 'exit':                                        
        # Adapta la pregunta al formato 
        image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)                                                
        question = txt_processors["eval"](question)
        #L anza pregunta al modelo           
        res = model.predict_answers(samples={"image": image, "text_input": question}, inference_method="generate")
        # Muestra respuesta
        print(res[0])
    else:
        Keepasking = False
  


#YOLO-World
https://www.ultralytics.com/es/blog/getting-hands-on-with-yolo-world
Hará AutoUpdate
permite incluso salvar un modelo para menor numero de clases
Sensibilidad

In [ ]:
from ultralytics import YOLOWorld

# Carga modelo preentrenado
model = YOLOWorld("yolov8s-world.pt")

# Clases de interés
model.set_classes(["car", "wheel", "car door", "car mirror", "license plate"])

# Predice las clases de interés
results = model.predict('images/Opel-Astra.png', max_det=100, iou=0.01, conf=0.01)

# Salva el resultado
results[0].save(filename='images/result.jpg')

# Salva el modelo adaptado a us necesidades
#model.save("custom_yolov8s.pt")